In [1]:
#0919: all questions and answers from topic "Jiankui He" 

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml import etree
from pyquery import PyQuery as pq
import pandas as pd
from pandas import Series, DataFrame
import urllib.request
import datetime
import time
import re
import csv

print("start.",datetime.datetime.now())
driver = webdriver.Chrome()
"""
#------------------------------------------------------------login
driver.get('http://www.quora.com/')
time.sleep(2)

form = driver.find_element_by_class_name('regular_login')
username = form.find_element_by_name('email')
username.send_keys('chenmeng0720@gmail.com')

password = form.find_element_by_name('password')
password.send_keys('321herosback')
time.sleep(1)

driver.find_element_by_xpath("//input[@class='submit_button ignore_interaction']").click()
time.sleep(2)

driver.find_element_by_link_text("Go back to English").click()
time.sleep(2)
"""
#------------------------------------------------------------1st stage,collect all questions links
url = "https://www.quora.com/topic/Dr-He-Jiankui/all_questions"    
       
driver.get(url)
driver.implicitly_wait(10)
time.sleep(2)

#scroll to the bottom at question page
soup = BeautifulSoup(driver.page_source,"lxml")
qun = len(soup.findAll("a", class_="question_link"))
qun2 = 0
i = 0
while qun > qun2:
    qun2 = qun
    for n in range(i*10,(i+1)*10):
        js = "var q = document.documentElement.scrollTop="+str(n*1000)
        driver.execute_script(js)
        time.sleep(1)
    soup = BeautifulSoup(driver.page_source,"lxml")
    qun = len(soup.findAll("a", class_="question_link"))   
    i+=1



soup = BeautifulSoup(driver.page_source,"lxml")
listql = []
for a in soup.findAll("a", class_="question_link"):
    b = a["href"]
    ql = "https://www.quora.com"+b
    if ql not in listql:
        listql.append(ql)

save = "/Users/CM/desktop/ScrapeZQ2/QClink/ql-drhjk.csv"
df = pd.DataFrame(listql,columns = ["QClink"])
df1 = df.to_csv(save, index = False)
print(len(listql),"questions acquired.")

print("done.",datetime.datetime.now())
driver.quit()

start. 2019-09-26 00:47:58.541406
1 questions acquired.
done. 2019-09-26 00:48:15.167373


In [2]:
#0923: collect all answer infos from question page
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml import etree
from pyquery import PyQuery as pq
import pandas as pd
from pandas import Series, DataFrame
import urllib.request
import datetime
import time
import re
import csv

print("start.",datetime.datetime.now())
driver = webdriver.Chrome()

#------------------------------------------------------------login and pre-work
driver.get('http://www.quora.com/')
time.sleep(2)

form = driver.find_element_by_class_name('regular_login')
username = form.find_element_by_name('email')
username.send_keys('chenmeng0720@gmail.com')

password = form.find_element_by_name('password')
password.send_keys('321herosback')
time.sleep(1)

driver.find_element_by_xpath("//input[@class='submit_button ignore_interaction']").click()
time.sleep(2)

driver.find_element_by_link_text("Go back to English").click()
time.sleep(2)

viewfooter = ["ContentFooter AnswerFooter ReadingContentFooter",
"ContentFooter ReadingContentFooter AnswerFooter",
"ReadingContentFooter ContentFooter AnswerFooter",
"ReadingContentFooter AnswerFooter ContentFooter",
"AnswerFooter ContentFooter ReadingContentFooter",
"AnswerFooter ReadingContentFooter ContentFooter"]

#------------------------------------------------------------circulation of getting answer details

with open("/Users/CM/desktop/ScrapeZQ2/qa-drhjk.csv","w",encoding = "utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["title","flwr","authlink","upno","share","cmtno","av","andate","imgno","textlen","antext"])
    df1 = pd.read_csv("/Users/CM/desktop/ScrapeZQ2/QClink/ql-drhjk.csv")["QClink"].values

    for qcl in df1:
        driver.get(qcl)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source,"lxml")
        
        try: 
            #get answer number 
            soup = BeautifulSoup(driver.page_source,"lxml")
            ann = soup.findAll("div", class_="QuestionPageAnswerHeader")
            #get question infos
            qtitle = soup.find("span", class_="rendered_qtext").get_text()
            qflwr = soup.find("span", class_="ui_button_count_inner").get_text()
            time.sleep(1)
            
            #scroll to the bottom at question page
            if len(ann) > 0:
                soup = BeautifulSoup(driver.page_source,"lxml")
                awn = len(soup.findAll("div", class_="pagedlist_item"))
                awn2 = 0
                i = 0
                while awn > awn2:
                    awn2 = awn
                    for n in range(i*5,(i+1)*5):
                        js = "var q = document.documentElement.scrollTop="+str(n*1000)
                        driver.execute_script(js)
                        time.sleep(0.6)
                    soup = BeautifulSoup(driver.page_source,"lxml")
                    awn = len(soup.findAll("div", class_="pagedlist_item"))   
                    i+=1
                
                #collect answer infos
                listmore = []
                soup = BeautifulSoup(driver.page_source,"lxml")
                ab = soup.find("div", class_="paged_list_wrapper")
                for an in ab.findAll("div", class_="pagedlist_item"):
                    
                    #non-anonymous,complete answers
                    if len(an.findAll("a",class_="ui_qtext_more_link")) == 0 and len(an.findAll("a",class_="user")) > 0:              
                        
                        authlink = "https://www.quora.com"+an.find("a",class_="user")["href"]
                        upno = an.find("span", class_="ui_button_count_optimistic_count u-block").get_text()
                        share = an.find("span", class_="ui_button_count_inner").get_text()
                        cmtno = str(len(an.findAll("div", class_="comment_wrapper comment_to_append")))+"个回复"
                        for vf in viewfooter:
                                if len(an.findAll("div", class_=vf)) > 0:
                                    av = an.find("div", class_=vf).get_text()
                                    break
                        andate = re.sub("Answered ","",an.find("a", class_="answer_permalink").get_text())
                        imgno = str(len(an.findAll("div", class_="ui_qtext_image_wrapper")))+"个图片"
                        antext = an.find("div", class_="ui_qtext_expanded").get_text()
                        textlen = str(len(antext))+"个字母"
                        
                        writer.writerow([qtitle,qflwr,authlink,upno,share,cmtno,av,andate,imgno,textlen,antext])
                        
                    #anonymous, complete answers
                    if len(an.findAll("a",class_="ui_qtext_more_link")) == 0 and len(an.findAll("span", class_="anon_user")) > 0:
                        
                        authlink = an.find("span", class_="anon_user").get_text()
                        upno = an.find("span", class_="ui_button_count_optimistic_count u-block").get_text()
                        share = an.find("span", class_="ui_button_count_inner").get_text()
                        cmtno = str(len(an.findAll("div", class_="comment_wrapper comment_to_append")))+"个回复"
                        for vf in viewfooter:
                                if len(an.findAll("div", class_=vf)) > 0:
                                    av = an.find("div", class_=vf).get_text()
                                    break
                        andate = re.sub("Answered ","",an.find("a", class_="answer_permalink").get_text())
                        imgno = str(len(an.findAll("div", class_="ui_qtext_image_wrapper")))+"个图片"
                        antext = an.find("div", class_="ui_qtext_expanded").get_text()
                        textlen = str(len(antext))+"个字母"
                        
                        writer.writerow([qtitle,qflwr,authlink,upno,share,cmtno,av,andate,imgno,textlen,antext])

                    #more link answers
                    if len(an.findAll("a",class_="ui_qtext_more_link")) != 0:
                        moli = "https://www.quora.com"+an.find("a",class_="ui_qtext_more_link")["href"]
                        if moli not in listmore:
                            listmore.append(moli)
                            
                #collect answer details of a morelink answer
                for moli in listmore:
                    driver.get(moli)
                    time.sleep(0.5)
                    
                    an = BeautifulSoup(driver.page_source,"lxml")
                    ah = an.find("div", class_="author_header")
                    #non-anonymous answer
                    if len(ah.findAll("a",class_="user")) > 0:
                    
                        authlink = "https://www.quora.com"+an.find("a",class_="user")["href"]
                        upno = an.find("span", class_="ui_button_count_optimistic_count u-block").get_text()
                        share = an.find("span", class_="ui_button_count_inner").get_text()
                        cmtno = str(len(an.findAll("div", class_="comment_wrapper last comment_to_append")))+"个回复"
                        for vf in viewfooter:
                                if len(an.findAll("div", class_=vf)) > 0:
                                    av = an.find("div", class_=vf).get_text()
                                    break
                        andate = re.sub("Answered ","",an.find("a", class_="answer_permalink").get_text())
                        imgno = str(len(an.findAll("div", class_="ui_qtext_image_wrapper")))+"个图片"
                        antext = an.find("div", class_="ui_qtext_expanded").get_text()
                        textlen = str(len(antext))+"个字母"
                        
                        writer.writerow([qtitle,qflwr,authlink,upno,share,cmtno,av,andate,imgno,textlen,antext])

                    #anonymous answer
                    if len(ah.findAll("span", class_="anon_user")) > 0:
                        
                        authlink = an.find("span", class_="anon_user").get_text()
                        upno = an.find("span", class_="ui_button_count_optimistic_count u-block").get_text()
                        share = an.find("span", class_="ui_button_count_inner").get_text()
                        cmtno = str(len(an.findAll("div", class_="comment_wrapper last comment_to_append")))+"个回复"
                        for vf in viewfooter:
                                if len(an.findAll("div", class_=vf)) > 0:
                                    av = an.find("div", class_=vf).get_text()
                                    break
                        andate = re.sub("Answered ","",an.find("a", class_="answer_permalink").get_text())
                        imgno = str(len(an.findAll("div", class_="ui_qtext_image_wrapper")))+"个图片"
                        antext = an.find("div", class_="ui_qtext_expanded").get_text()
                        textlen = str(len(antext))+"个字母"
                        
                        writer.writerow([qtitle,qflwr,authlink,upno,share,cmtno,av,andate,imgno,textlen,antext])
                        
            else:
                writer.writerow([qtitle,qflwr])
                
        except:
            print(qcl) 
            #fill up manually

print("done.",datetime.datetime.now())
time.sleep(1)
driver.quit()
        

start. 2019-09-26 00:48:48.319225
done. 2019-09-26 00:49:15.694882


In [3]:
#0923: answerers detail
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml import etree
from pyquery import PyQuery as pq
import pandas as pd
from pandas import Series, DataFrame
import urllib.request
import datetime
import time
import re
import csv

print("start.",datetime.datetime.now())
driver = webdriver.Chrome()

#------------------------------------------------------------login and pre-work
driver.get('http://www.quora.com/')
time.sleep(2)

form = driver.find_element_by_class_name('regular_login')
username = form.find_element_by_name('email')
username.send_keys('chenmeng0720@gmail.com')

password = form.find_element_by_name('password')
password.send_keys('321herosback')
time.sleep(1)

driver.find_element_by_xpath("//input[@class='submit_button ignore_interaction']").click()
time.sleep(2)

driver.find_element_by_link_text("Go back to English").click()
time.sleep(2)

#------------------------------------------------------------get answerer details

with open("/Users/CM/desktop/ScrapeZQ2/ai-drhjk.csv","w",encoding = "utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["authlink","authname","influencer","authflwr","authtv","authmv","awno"])
    df1 = pd.read_csv("/Users/CM/desktop/ScrapeZQ2/qa-drhjk.csv")["authlink"].values
    #df1 = ["https://www.quora.com/profile/Barack-Obama-44"]
    
    listqcl = []
    for qcl in df1:
        qcl = str(qcl)
        if "https" in qcl and qcl not in listqcl:
            listqcl.append(qcl)
            try:  
                driver.get(qcl)
                time.sleep(0.2)
                soup = BeautifulSoup(driver.page_source,"lxml")
                
                authname = soup.find("span", class_="user").get_text()
                
                if len(soup.findAll("span", class_="VerificationBadge")) > 0 or len(soup.findAll("div", class_="TopWriterAboutListItem AboutListItem")) > 0:
                    influencer = "1"
                else:
                    influencer = "0"
                authflwr = re.sub("Followers ","",soup.find("a", href=re.sub("^https://www.quora.com","",qcl+"/followers")).get_text())
                awno = re.sub("Answers","",soup.find("a", href=re.sub("^https://www.quora.com","",qcl+"/answers")).get_text())
                #authtv = re.sub(" content views","",soup.find("div", class_="AboutListItem ContentViewsAboutListItem").find("span", class_="body_text").find("span", class_="main_text").get_text())
                #authmv = re.sub(" this month","",soup.find("div", class_="AboutListItem ContentViewsAboutListItem").find("span", class_="body_text").find("span", class_="detail_text").get_text())
                for n in range(10):
                    if len(soup.findAll("div", class_="AboutListItem ContentViewsAboutListItem")) > 0:
                        authtv = re.sub(" content views","",soup.find("div", class_="AboutListItem ContentViewsAboutListItem").find("span", class_="body_text").find("span", class_="main_text").get_text())
                        authmv = re.sub(" this month","",soup.find("div", class_="AboutListItem ContentViewsAboutListItem").find("span", class_="body_text").find("span", class_="detail_text").get_text())
                        break
                    else:
                        driver.get(qcl)
                        soup = BeautifulSoup(driver.page_source,"lxml")
                        authtv = 0
                        authmv = 0
                writer.writerow([qcl,authname,influencer,authflwr,authtv,authmv,awno])
                time.sleep(0.5)
            except:
                print(qcl)
                #fill up manually

print("done.",datetime.datetime.now())
driver.quit()




start. 2019-09-26 00:56:24.284046
done. 2019-09-26 00:56:53.089238
